In [1]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


from sklearn.model_selection import TimeSeriesSplit
from sklearn import preprocessing
from scipy import stats

#import pyarrow as pa
#import pyarrow.parquet as pq

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
plt.style.use('ggplot')

In [2]:
## Complete Case Analysis , completely removes missing value rows if the make up less than 3 percent of the dataset
def CCA(df):
    col=df.columns
    for i in range(col.size):
                   if ((df[col[i]].isnull().mean()*100) < 3):
                            df=df.dropna(axis='index', subset=[col[i]])
               
 
    return df  

In [3]:
def freqEncoding(df,l):
    df_copy=df.copy()
    for i in range (len(l)):
        enc=(df_copy.groupby(df_copy[l[i]]).size()) / len(df_copy)
        df_copy[l[i]] = df_copy[l[i]].apply(lambda x : enc[x])
        df_enc=pd.DataFrame(enc)
        #df_enc=swapRows(df_enc,l[i])
        df_enc.to_csv('encodedLabels.csv', mode='a')
    return df_copy

def labelEncoding(df,l):
    result = df.copy() 
    for i in range(len(l)):
            le=preprocessing.LabelEncoder()
            result[l[i]] = le.fit_transform(result[l[i]])
            encodingKey=le.classes_
            df_encodedKey=pd.DataFrame (encodingKey, columns = [l[i]])
            df_encodedKey.to_csv('encodedLabels.csv', mode='a')#this line adds the encoded keys for lookup 
    return result


In [4]:
df_accidents=pd.read_csv('2017_Accidents_UK.csv')
df_accidents.head(50)

,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
0,2017010001708,2017,10001708,532920.0,196330.0,-0.080107,51.650061,Metropolitan Police,Fatal,2,3,05/08/2017,Saturday,03:12,Enfield,Enfield,Enfield,A,105,Single carriageway,30.0,Not at junction or within 20 metres,Data missing or out of range,-1,NaN,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Dry,None,None,Urban,Yes,Non-trunk,E01001450
1,2017010009342,2017,10009342,526790.0,181970.0,-0.173845,51.522425,Metropolitan Police,Slight,2,1,01/01/2017,Sunday,01:30,Westminster,Westminster,Westminster,A,5,Single carriageway,30.0,T or staggered junction,Give way or uncontrolled,Unclassified,first_road_class is C or Unclassified. These r...,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Wet or damp,None,None,Urban,Yes,Non-trunk,E01004702
2,2017010009344,2017,10009344,535200.0,181260.0,-0.052969,51.514096,Metropolitan Police,Slight,3,1,01/01/2017,Sunday,00:30,Tower Hamlets,Tower Hamlets,Tower Hamlets,A,13,Single carriageway,30.0,T or staggered junction,Give way or uncontrolled,C,first_road_class is C or Unclassified. These r...,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Dry,None,None,Urban,Yes,Non-trunk,E01004298
3,2017010009348,2017,10009348,534340.0,193560.0,-0.060658,51.624832,Metropolitan Police,Slight,2,1,01/01/2017,Sunday,01:11,Enfield,Enfield,Enfield,A,1010,Roundabout,30.0,Roundabout,Give way or uncontrolled,B,154.0,None within 50 metres,"Pelican, puffin, toucan or similar non-junctio...",Darkness - lights lit,Raining no high winds,Wet or damp,None,None,Urban,Yes,Non-trunk,E01001429
4,2017010009350,2017,10009350,533680.0,187820.0,-0.072372,51.573408,Metropolitan Police,Serious,1,1,01/01/2017,Sunday,01:42,Hackney,Hackney,Hackney,A,107,Dual carriageway,20.0,Crossroads,Auto traffic signal,A,10.0,None within 50 metres,Pedestrian phase at traffic signal junction,Darkness - lights lit,Fine no high winds,Wet or damp,None,None,Urban,Yes,Non-trunk,E01001808
5,2017010009351,2017,10009351,514510.0,172370.0,-0.353876,51.438762,Metropolitan Police,Slight,2,1,01/01/2017,Sunday,03:31,Richmond upon Thames,Richmond upon Thames,Richmond upon Thames,Unclassified,first_road_class is C or Unclassified. These r...,Single carriageway,30.0,Not at junction or within 20 metres,Data missing or out of range,-1,NaN,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Wet or damp,None,None,Urban,Yes,Non-trunk,E01003900
6,2017010009353,2017,10009353,508640.0,181870.0,-0.435377,51.525305,Metropolitan Police,Slight,2,2,01/01/2017,Sunday,04:07,Hillingdon,Hillingdon,Hillingdon,A,4020,Dual carriageway,40.0,T or staggered junction,Give way or uncontrolled,C,first_road_class is C or Unclassified. These r...,None within 50 metres,"Pelican, puffin, toucan or similar non-junctio...",Darkness - lights lit,Fine no high winds,Wet or damp,None,None,Urban,Yes,Non-trunk,E01002401
7,2017010009354,2017,10009354,527880.0,181950.0,-0.158150,51.522000,Metropolitan Police,Slight,2,1,01/01/2017,Sunday,05:20,Westminster,Westminster,Westminster,A,501,Dual carriageway,30.0,Not at junction or within 20 metres,Data missing or out of range,-1,NaN,Cont

In [15]:
df_accidents['accident_severity'].value_counts()

Slight     105950
Serious     22356
Fatal        1676
Name: accident_severity, dtype: int64

In [5]:
df_accidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129982 entries, 0 to 129981
Data columns (total 36 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   accident_index                               129982 non-null  object 
 1   accident_year                                129982 non-null  int64  
 2   accident_reference                           129982 non-null  object 
 3   location_easting_osgr                        129963 non-null  float64
 4   location_northing_osgr                       129963 non-null  float64
 5   longitude                                    129953 non-null  float64
 6   latitude                                     129953 non-null  float64
 7   police_force                                 129982 non-null  object 
 8   accident_severity                            129982 non-null  object 
 9   number_of_vehicles                           129982 non-nul

In [6]:
df_accidents.isnull().sum() ## shows the number of missing data in each column

accident_index                                     0
accident_year                                      0
accident_reference                                 0
location_easting_osgr                             19
location_northing_osgr                            19
longitude                                         29
latitude                                          29
police_force                                       0
accident_severity                                  0
number_of_vehicles                                 0
number_of_casualties                               0
date                                               0
day_of_week                                        0
time                                               0
local_authority_district                           0
local_authority_ons_district                       0
local_authority_highway                            0
first_road_class                                   0
first_road_number                             

In [7]:
df_accidentsClean=CCA(df_accidents) ## performs complete case analysis on the dataset

In [8]:
# shows the number of missing data in each column after CCA , 
#clearly all are removed but "second_road_number" since the missing rows make up more than 3% of the dataset
df_accidentsClean.isnull().sum()

accident_index                                     0
accident_year                                      0
accident_reference                                 0
location_easting_osgr                              0
location_northing_osgr                             0
longitude                                          0
latitude                                           0
police_force                                       0
accident_severity                                  0
number_of_vehicles                                 0
number_of_casualties                               0
date                                               0
day_of_week                                        0
time                                               0
local_authority_district                           0
local_authority_ons_district                       0
local_authority_highway                            0
first_road_class                                   0
first_road_number                             

In [9]:
#Imputes all missing "second_road_number" rows with "Missing"
df_accidentsClean['second_road_number']=df_accidentsClean['second_road_number'].replace(np.nan,"Missing")
df_accidentsClean['second_road_number'].isnull().sum() 

0

In [10]:
#performs frequncy encoding on the following columns:
l=['day_of_week','police_force','light_conditions','local_authority_district','local_authority_ons_district','local_authority_highway' \
  ,'weather_conditions','road_type' ,'junction_detail','junction_control','road_surface_conditions','special_conditions_at_site',\
   'carriageway_hazards','urban_or_rural_area','trunk_road_flag','did_police_officer_attend_scene_of_accident','pedestrian_crossing_physical_facilities','pedestrian_crossing_human_control','second_road_number','first_road_number']
df_accidentsClean=freqEncoding(df_accidentsClean,l)
df_accidentsClean.head()

,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
0,2017010001708,2017,10001708,532920.0,196330.0,-0.080107,51.650061,0.197239,Fatal,2,3,05/08/2017,0.132719,03:12,0.005989,0.005989,0.005989,A,0.001616,0.735379,30.0,0.416478,0.418434,-1,0.417577,0.977588,0.795293,0.207657,0.835243,0.718221,0.973215,0.976513,0.666866,0.758745,0.838153,E01001450
1,2017010009342,2017,10009342,526790.0,181970.0,-0.173845,51.522425,0.197239,Slight,2,1,01/01/2017,0.115059,01:30,0.012536,0.012536,0.012536,A,0.006312,0.735379,30.0,0.300837,0.454820,Unclassified,0.440305,0.977588,0.795293,0.207657,0.835243,0.256999,0.973215,0.976513,0.666866,0.758745,0.838153,E01004702
2,2017010009344,2017,10009344,535200.0,181260.0,-0.052969,51.514096,0.197239,Slight,3,1,01/01/2017,0.115059,00:30,0.008341,0.008341,0.008341,A,0.003459,0.735379,30.0,0.300837,0.454820,C,0.440305,0.977588,0.795293,0.207657,0.835243,0.718221,0.973215,0.976513,0.666866,0.758745,0.838153,E01004298
3,2017010009348,2017,10009348,534340.0,193560.0,-0.060658,51.624832,0.197239,Slight,2,1,01/01/2017,0.115059,01:11,0.005989,0.005989,0.005989,A,0.001245,0.066000,30.0,0.083813,0.454820,B,0.000081,0.977588,0.062258,0.207657,0.114146,0.256999,0.973215,0.976513,0.666866,0.758745,0.838153,E01001429
4,2017010009350,2017,10009350,533680.0,187820.0,-0.072372,51.573408,0.197239,Serious,1,1,01/01/2017,0.115059,01:42,0.006918,0.006918,0.006918,A,0.001350,0.160804,20.0,0.095969,0.111042,A,0.000921,0.977588,0.073120,0.207657,0.835243,0.256999,0.973215,0.976513,0.666866,0.758745,0.838153,E01001808


In [11]:
#performs label encoding on the following columns:
l=['accident_severity','first_road_class','second_road_class']
df_accidentsClean=labelEncoding(df_accidentsClean,l)
df_accidentsClean.head()

,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
0,2017010001708,2017,10001708,532920.0,196330.0,-0.080107,51.650061,0.197239,0,2,3,05/08/2017,0.132719,03:12,0.005989,0.005989,0.005989,0,0.001616,0.735379,30.0,0.416478,0.418434,0,0.417577,0.977588,0.795293,0.207657,0.835243,0.718221,0.973215,0.976513,0.666866,0.758745,0.838153,E01001450
1,2017010009342,2017,10009342,526790.0,181970.0,-0.173845,51.522425,0.197239,2,2,1,01/01/2017,0.115059,01:30,0.012536,0.012536,0.012536,0,0.006312,0.735379,30.0,0.300837,0.454820,6,0.440305,0.977588,0.795293,0.207657,0.835243,0.256999,0.973215,0.976513,0.666866,0.758745,0.838153,E01004702
2,2017010009344,2017,10009344,535200.0,181260.0,-0.052969,51.514096,0.197239,2,3,1,01/01/2017,0.115059,00:30,0.008341,0.008341,0.008341,0,0.003459,0.735379,30.0,0.300837,0.454820,4,0.440305,0.977588,0.795293,0.207657,0.835243,0.718221,0.973215,0.976513,0.666866,0.758745,0.838153,E01004298
3,2017010009348,2017,10009348,534340.0,193560.0,-0.060658,51.624832,0.197239,2,2,1,01/01/2017,0.115059,01:11,0.005989,0.005989,0.005989,0,0.001245,0.066000,30.0,0.083813,0.454820,3,0.000081,0.977588,0.062258,0.207657,0.114146,0.256999,0.973215,0.976513,0.666866,0.758745,0.838153,E01001429
4,2017010009350,2017,10009350,533680.0,187820.0,-0.072372,51.573408,0.197239,1,1,1,01/01/2017,0.115059,01:42,0.006918,0.006918,0.006918,0,0.001350,0.160804,20.0,0.095969,0.111042,1,0.000921,0.977588,0.073120,0.207657,0.835243,0.256999,0.973215,0.976513,0.666866,0.758745,0.838153,E01001808


In [12]:
df_accidentsClean['accident_severity'].info()

<class 'pandas.core.series.Series'>
Int64Index: 123728 entries, 0 to 129981
Series name: accident_severity
Non-Null Count   Dtype
--------------   -----
123728 non-null  int64
dtypes: int64(1)
memory usage: 1.9 MB


In [13]:
#the encoding functions above add  all labels with their corresponding new encoded values for lookup 
df_labels=pd.read_csv('encodedLabels.csv')
df_labels.head()

,day_of_week,0
0,Friday,0.1632290184921764
1,Monday,0.13831953963532911
2,Saturday,0.13271854390275442
3,Sunday,0.11505883874304927
4,Thursday,0.15203510927195138


In [14]:
df_accidentsClean.to_csv('2017_Accidents_UK_Clean.csv')#extracts the clean dataset to a new csv file